# ***Car Prediction Model using Mobilenet***
This code aims to create and train model for car prediction. Here the transfer learning technique is used create the model.

In this new update we have increased the augmentation and unfrozed more layers. while implementing the same model with a bit high learning rate.

**Libraries**

In [ ]:
#Imporatant Libraries for working with arrays.
import numpy as np

#Important Libraries for visulization of the results.
import matplotlib.pyplot as plt

#Important Libraries for training and testing of the model.
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input

#configuring the gpu 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
tf.version
assert tf.test.is_built_with_cuda()
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)


**Model Generator**

In [ ]:
def Model_Generator(IMAGE_SIZE):

  #Loading the model
  model=tf.keras.models.load_model('D:/car196/car100/New_model_100_classes_V2.h5')
  #freezing all layers
  for layer in model.layers:
    layer.trainable = False

  #Unfreezing last 30 layer

  for layer in model.layers[-50:]:
    layer.trainable = True

  #Printting Information about the model.
  print(model.summary())
  return model

**Optimizer generator**

In [ ]:
def Optimizer_Generator():
  # Set the parameters value for optimizer
  adam=tf.keras.optimizers.Adam(
          learning_rate=1e-4,
          beta_1=0.9,
          beta_2=0.999,
          epsilon=1e-07,
          decay = 0.0
  )
  return adam

**Model training**

In [ ]:
def Model_training(training_set, test_set, callback):
  results = model.fit(
                        training_set,
                        validation_data=test_set,
                        epochs=15,
                        steps_per_epoch=len(training_set),
                        validation_steps=len(test_set),
                        callbacks = [callback]
                      )
  return results

In [ ]:
#Dimension of input image.  
Image_Dimension = [227,227]

#Generation of the model
model = Model_Generator(Image_Dimension)

#Generation of the optimizer
adam = Optimizer_Generator()

#Compile model.
model.compile(
  loss='categorical_crossentropy',
  optimizer=adam,
  metrics=['accuracy']
)

#Generating flow from the directories for training and testing dataset.

#Genrating flow of images from training dataset.
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range = 30,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('D:/car196/car100/training',
                                                 target_size = (227, 227),
                                                 batch_size = 256,
                                                 class_mode = 'categorical',
                                                 shuffle=True)

#Genrating flow of images from testing dataset.
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_set = test_datagen.flow_from_directory('D:/car196/car100/testing',
                                            target_size = (227, 227),
                                            batch_size = 256,
                                            class_mode = 'categorical',
                                            shuffle=True)

#Set Callback options here.
checkpoint_filepath = 'D:/car196/car100/weights.{epoch:02d}val_acc{val_accuracy:3f}.hdf5'
callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4,mode = 'auto', restore_best_weights=False),
            tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.1, patience=2, mode="auto",min_delta=0.0001,min_lr=0),
            tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True),
           ]


results = Model_training(training_set, test_set, callback)

#Visualizing the results.

#Plot for epoch versus loss.
plt.plot(results.history['loss'], label='train loss')
plt.plot(results.history['val_loss'], label='val loss')
plt.legend()
plt.show()

#Plot for epoch versus accuracy.
plt.plot(results.history['accuracy'], label='train acc')
plt.plot(results.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
 
#Save model 
model.save('D:/car196/car100/New_model_100_classes_V3.h5')